# star schema 

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

# Creating Dimension Tables and Fact Table

In [3]:
%%sql
--dimension table for course catalog 
-- Course Catalogs table
DROP TABLE IF EXISTS CATALOG_COURSES_DIM;
CREATE TABLE CATALOG_COURSES_DIM (
    CourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    ProgramID INTEGER,
    CourseTitle TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT
    
);


 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [4]:
%%sql
-- dimension table for program 
-- Programs table as Dimension_Table
DROP TABLE IF EXISTS PROGRAMS_DIM;
CREATE TABLE PROGRAMS_DIM (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [5]:
%%sql
--dimension table for location

-- Locations table
DROP TABLE IF EXISTS LOCATIONS_DIM;
CREATE TABLE LOCATIONS_DIM (
    LocationID INTEGER PRIMARY KEY,
    LocationCode TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [6]:
%%sql
--dimension table for instructor
-- Instructors table
DROP TABLE IF EXISTS INSTRUCTORS_DIM;
CREATE TABLE INSTRUCTORS_DIM (
    InstructorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);


 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [7]:
%%sql 
-- Fact Table 
DROP TABLE IF EXISTS COURSE_MEETING_FACTS;
CREATE TABLE COURSE_MEETING_FACTS(
    CourseMeetingID INTEGER PRIMARY KEY,
    CourseOfferingID INTEGER NOT NULL,
    LocationID INTEGER NOT NULL,
    StartDateTime TEXT NOT NULL,
    EndDateTime TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    CourseID INTEGER NOT NULL,
    ProgramID INTEGER NOT NULL,
    CourseTitle TEXT NOT NULL,
    CatalogYear TEXT NOT NULL,
    PrimaryInstructorID INTEGER NOT NULL,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    Section TEXT NOT NULL,
    Credits REAL,
    Title TEXT NOT NULL,
    Timecodes TEXT
);

    

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

- roll up queries 


# Loading Data

In [8]:
%%sql
ATTACH DATABASE 'CourseData.db' as Course_Info;


 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) database is locked
[SQL: ATTACH DATABASE 'CourseData.db' as Course_Info;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [9]:
%%sql
SELECT * 
FROM Course_Info.PROGRAMS;


 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.PROGRAMS
[SQL: SELECT * 
FROM Course_Info.PROGRAMS;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [10]:
%%sql
SELECT * FROM PROGRAMS;
PRAGMA Course_Info.PROGRAMS;

 * sqlite:///CourseDataWarehouse.db
Done.
(sqlite3.OperationalError) unknown database Course_Info
[SQL: PRAGMA Course_Info.PROGRAMS;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [11]:
%%sql
SELECT * 
FROM Course_Info.Programs;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.Programs
[SQL: SELECT * 
FROM Course_Info.Programs;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [12]:
%%sql
SELECT DISTINCT CATALOGID, TERM, CRN, TITLE
FROM Course_Info.INSTRUCTORS
    JOIN Course_Info.COURSE_OFFERINGS ON (Course_Info.INSTRUCTORS.INSTRUCTORID=Course_Info.COURSE_OFFERINGS.PRIMARYINSTRUCTORID)
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.INSTRUCTORS
[SQL: SELECT DISTINCT CATALOGID, TERM, CRN, TITLE
FROM Course_Info.INSTRUCTORS
    JOIN Course_Info.COURSE_OFFERINGS ON (Course_Info.INSTRUCTORS.INSTRUCTORID=Course_Info.COURSE_OFFERINGS.PRIMARYINSTRUCTORID)
LIMIT 5;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [13]:
%%sql
SELECT *
FROM Course_Info.LOCATIONS
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.LOCATIONS
[SQL: SELECT *
FROM Course_Info.LOCATIONS
LIMIT 5;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [14]:
%%sql
SELECT PRIMARYINSTRUCTORID, COUNT(DISTINCT COURSEID) as COURSECOUNT
FROM Course_Info.COURSE_OFFERINGS
GROUP BY PRIMARYINSTRUCTORID
HAVING COUNT(*) = 1
LIMIT 5;


 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.COURSE_OFFERINGS
[SQL: SELECT PRIMARYINSTRUCTORID, COUNT(DISTINCT COURSEID) as COURSECOUNT
FROM Course_Info.COURSE_OFFERINGS
GROUP BY PRIMARYINSTRUCTORID
HAVING COUNT(*) = 1
LIMIT 5;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [15]:
%%sql
--how many people enrolled in ba0500
SELECT DISTINCT CATALOGID, TERM, ACTUAL, TITLE, COUNT(DISTINCT COURSEID) as COURSECOUNT
FROM Course_Info.Course_Offerings 
WHERE CATALOGID = 'BA 0500'


 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.Course_Offerings
[SQL: --how many people enrolled in ba0500
SELECT DISTINCT CATALOGID, TERM, ACTUAL, TITLE, COUNT(DISTINCT COURSEID) as COURSECOUNT
FROM Course_Info.Course_Offerings 
WHERE CATALOGID = 'BA 0500']
(Background on this error at: http://sqlalche.me/e/e3q8)


In [16]:
%%sql
--which professors teach ethics 
SELECT CATALOGID, CRN, SECTION, TITLE, PRIMARYINSTRUCTORID
FROM Course_Info.Course_Offerings
    JOIN INSTRUCTORS_DIM ON (INSTRUCTORS_DIM.Name = Course_Info.COURSE_OFFERINGS.PrimaryInstructorID)
WHERE CATALOGID = 'BU 0391';

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Course_Info.Course_Offerings
[SQL: --which professors teach ethics 
SELECT CATALOGID, CRN, SECTION, TITLE, PRIMARYINSTRUCTORID
FROM Course_Info.Course_Offerings
    JOIN INSTRUCTORS_DIM ON (INSTRUCTORS_DIM.Name = Course_Info.COURSE_OFFERINGS.PrimaryInstructorID)
WHERE CATALOGID = 'BU 0391';]
(Background on this error at: http://sqlalche.me/e/e3q8)
